In [1]:
import time
import pandas as pd
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import urllib3
import requests

In [2]:
# Call the dataset
mdpi_links = pd.read_excel("./query-links/MDPI_Query_Links.xlsx")

In [4]:
# Open the drive
driver = uc.Chrome()

In [ ]:
# Get the paper links
list_source = []
list_querystring = []
list_querylinks = []
list_paperlinks = []
for index in range(0,len(mdpi_links["Query Links"])):
  driver.get(mdpi_links["Query Links"][index])
  time.sleep(5)
  try:
      driver.find_element_by_link_text('Accept').send_keys('\n')
      time.sleep(5)
  except:
      pass
  try:
    # Get links from the first page
    page = requests.get(mdpi_links["Query Links"][index]) # URL given by the query generator
    soup = BeautifulSoup(page.text,'html.parser')
    info = soup.select("[class~=title-link]")
    [list_source.append("MDPI") for i in info]
    [list_querystring.append(mdpi_links["Query"][index]) for i in info]
    [list_querylinks.append(mdpi_links["Query Links from Abstract"][index]) for i in info]
    [list_paperlinks.append("https://www.mdpi.com"+i.get('href')) for i in info]

    # Get links from the next page
    while True:
      try:
        driver.find_element_by_link_text('chevron_right').send_keys('\n')
        time.sleep(5)
        next_url = driver.current_url
        page = requests.get(next_url) # URL given by the query generator
        soup = BeautifulSoup(page.text,'html.parser')
        info = soup.select("[class~=title-link]")
        [list_source.append("MDPI") for i in info]
        [list_querystring.append(mdpi_links["Query"][index]) for i in info]
        [list_querylinks.append(mdpi_links["Query Links"][index]) for i in info]
        [list_paperlinks.append("https://www.mdpi.com"+i.get('href')) for i in info]
      except:
        break
  except:
    list_source.append("MDPI")
    list_querystring.append(mdpi_links["Query"][index])
    list_querylinks.append(mdpi_links["Query Links"][index])
    list_paperlinks.append("no_result")

In [ ]:
# Save to xlsx
mdpi_paperlinks = pd.DataFrame({'Source': list_source,'Query': list_querystring, 'Query Links': list_querylinks, 'Paper Links': list_paperlinks})
mdpi_paperlinks.to_excel("./paper-links/PaperLinks_MDPI_AllDuplicated.xlsx", index = False)
len(mdpi_paperlinks)

In [ ]:
# Drop Duplicate
mdpi_paperlinks_dd = mdpi_paperlinks.drop_duplicates(['Paper Links'], keep='first')
mdpi_paperlinks_dd.to_excel("./paper-links/PaperLinks_MDPI.xlsx", index = False)
len(mdpi_paperlinks_dd)